# Simple Baseline Models for S&P 500 Prediction

## Overview
This notebook implements baseline models with proper model selection using the original train.csv and test.csv files.

**Approach:**
- Use original data without advanced imputation
- Implement robust model selection with cross-validation
- Train on train.csv and evaluate on test.csv
- Focus on interpretable baseline performance
- Include competition-specific metrics

**Models:**
- Linear models: Ridge, Lasso, ElasticNet
- Tree-based: RandomForest, GradientBoosting
- Advanced: XGBoost, LightGBM

## 1. Setup and Data Loading

In [1]:
# Import essential libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Machine learning libraries
from sklearn.model_selection import train_test_split, cross_val_score, TimeSeriesSplit
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.impute import SimpleImputer

# Advanced models (with fallback)
try:
    import xgboost as xgb
    import lightgbm as lgb
    ADVANCED_MODELS = True
    print("✅ Advanced models (XGBoost, LightGBM) available")
except ImportError:
    ADVANCED_MODELS = False
    print("⚠️ Advanced models not available - using sklearn models only")

# Configure plotting
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

print("✅ Libraries imported successfully")

✅ Advanced models (XGBoost, LightGBM) available
✅ Libraries imported successfully


In [2]:
# Load training and test data
print("📊 Loading datasets...")
df_train = pd.read_csv('../data/raw/train.csv')
df_test = pd.read_csv('../data/raw/test.csv')

print(f"✅ Data loaded successfully:")
print(f"   • Training data: {df_train.shape}")
print(f"   • Test data: {df_test.shape}")
print(f"   • Training missing values: {df_train.isnull().sum().sum():,}")
print(f"   • Test missing values: {df_test.isnull().sum().sum():,}")

# Check for target variable
target_cols = ['forward_returns', 'market_forward_excess_returns', 'risk_free_rate']
available_targets = [col for col in target_cols if col in df_train.columns]
print(f"   • Available targets: {available_targets}")

if 'forward_returns' in df_train.columns:
    primary_target = 'forward_returns'
    print(f"   • Using primary target: {primary_target}")
else:
    primary_target = available_targets[0] if available_targets else None
    print(f"   • Using fallback target: {primary_target}")

📊 Loading datasets...
✅ Data loaded successfully:
   • Training data: (8990, 98)
   • Test data: (10, 99)
   • Training missing values: 137,675
   • Test missing values: 0
   • Available targets: ['forward_returns', 'market_forward_excess_returns', 'risk_free_rate']
   • Using primary target: forward_returns
✅ Data loaded successfully:
   • Training data: (8990, 98)
   • Test data: (10, 99)
   • Training missing values: 137,675
   • Test missing values: 0
   • Available targets: ['forward_returns', 'market_forward_excess_returns', 'risk_free_rate']
   • Using primary target: forward_returns


## 2. Data Preprocessing

In [3]:
print("🔧 DATA PREPROCESSING")
print("=" * 50)

# Identify feature columns (exclude targets and IDs)
exclude_cols = ['date_id', 'forward_returns', 'market_forward_excess_returns', 'risk_free_rate']
feature_cols = [col for col in df_train.columns if col not in exclude_cols]

print(f"📊 Feature Analysis:")
print(f"   • Total features: {len(feature_cols)}")
print(f"   • Excluded columns: {exclude_cols}")

# Analyze missing values in features
missing_analysis = pd.DataFrame({
    'column': feature_cols,
    'missing_count': [df_train[col].isnull().sum() for col in feature_cols],
    'missing_pct': [df_train[col].isnull().sum() / len(df_train) * 100 for col in feature_cols]
})
missing_analysis = missing_analysis.sort_values('missing_pct')

# Categorize features by missing value percentage
complete_features = missing_analysis[missing_analysis['missing_pct'] == 0]['column'].tolist()
low_missing = missing_analysis[(missing_analysis['missing_pct'] > 0) & (missing_analysis['missing_pct'] <= 10)]['column'].tolist()
medium_missing = missing_analysis[(missing_analysis['missing_pct'] > 10) & (missing_analysis['missing_pct'] <= 50)]['column'].tolist()
high_missing = missing_analysis[missing_analysis['missing_pct'] > 50]['column'].tolist()

print(f"\n📈 Missing Value Categories:")
print(f"   • Complete features (0% missing): {len(complete_features)}")
print(f"   • Low missing (1-10%): {len(low_missing)}")
print(f"   • Medium missing (11-50%): {len(medium_missing)}")
print(f"   • High missing (>50%): {len(high_missing)}")

# Show complete features (most reliable for baseline)
if len(complete_features) > 0:
    print(f"\n✅ Complete features: {complete_features[:10]}{'...' if len(complete_features) > 10 else ''}")
else:
    print(f"\n⚠️ No complete features found - will use simple imputation")

🔧 DATA PREPROCESSING
📊 Feature Analysis:
   • Total features: 94
   • Excluded columns: ['date_id', 'forward_returns', 'market_forward_excess_returns', 'risk_free_rate']

📈 Missing Value Categories:
   • Complete features (0% missing): 9
   • Low missing (1-10%): 0
   • Medium missing (11-50%): 77
   • High missing (>50%): 8

✅ Complete features: ['D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9']


In [4]:
# Create different feature sets for model selection
print("\n🎯 FEATURE SET CREATION")
print("=" * 50)

# Strategy 1: Use only complete features
feature_sets = {}

if len(complete_features) >= 5:
    feature_sets['complete_only'] = complete_features
    print(f"✅ Feature Set 1: Complete features only ({len(complete_features)} features)")

# Strategy 2: Include low missing value features with imputation
reliable_features = complete_features + low_missing
if len(reliable_features) >= 5:
    feature_sets['reliable'] = reliable_features
    print(f"✅ Feature Set 2: Reliable features (≤10% missing) ({len(reliable_features)} features)")

# Strategy 3: Include medium missing with imputation
extended_features = complete_features + low_missing + medium_missing
if len(extended_features) >= 5:
    feature_sets['extended'] = extended_features
    print(f"✅ Feature Set 3: Extended features (≤50% missing) ({len(extended_features)} features)")

# Strategy 4: All features with aggressive imputation
if len(feature_cols) >= 5:
    feature_sets['all_features'] = feature_cols
    print(f"✅ Feature Set 4: All features with imputation ({len(feature_cols)} features)")

# Fallback: Use top features by correlation if no good sets
if len(feature_sets) == 0:
    print("⚠️ Creating fallback feature set based on correlation...")
    
    # Simple correlation analysis with target
    if primary_target in df_train.columns:
        target_corr = df_train[feature_cols + [primary_target]].corr()[primary_target].abs().sort_values(ascending=False)
        top_corr_features = target_corr.drop(primary_target).head(10).index.tolist()
        feature_sets['top_correlated'] = top_corr_features
        print(f"✅ Fallback: Top correlated features ({len(top_corr_features)} features)")

print(f"\n📊 Created {len(feature_sets)} feature sets for model selection")
for name, features in feature_sets.items():
    print(f"   • {name}: {len(features)} features")


🎯 FEATURE SET CREATION
✅ Feature Set 1: Complete features only (9 features)
✅ Feature Set 2: Reliable features (≤10% missing) (9 features)
✅ Feature Set 3: Extended features (≤50% missing) (86 features)
✅ Feature Set 4: All features with imputation (94 features)

📊 Created 4 feature sets for model selection
   • complete_only: 9 features
   • reliable: 9 features
   • extended: 86 features
   • all_features: 94 features


## 3. Model Setup and Configuration

In [5]:
# Define baseline models
print("🤖 MODEL CONFIGURATION")
print("=" * 50)

# Configure models with reasonable parameters
models = {
    'Ridge': Ridge(alpha=1.0, random_state=42),
    'Lasso': Lasso(alpha=0.1, random_state=42, max_iter=2000),
    'ElasticNet': ElasticNet(alpha=0.1, l1_ratio=0.5, random_state=42, max_iter=2000),
    'RandomForest': RandomForestRegressor(
        n_estimators=100, 
        max_depth=10, 
        random_state=42, 
        n_jobs=-1
    ),
    'GradientBoosting': GradientBoostingRegressor(
        n_estimators=100, 
        learning_rate=0.1, 
        max_depth=6, 
        random_state=42
    )
}

# Add advanced models if available
if ADVANCED_MODELS:
    models['XGBoost'] = xgb.XGBRegressor(
        n_estimators=100,
        learning_rate=0.1,
        max_depth=6,
        random_state=42,
        verbosity=0
    )
    models['LightGBM'] = lgb.LGBMRegressor(
        n_estimators=100,
        learning_rate=0.1,
        max_depth=6,
        random_state=42,
        verbosity=-1
    )

print(f"✅ Configured {len(models)} baseline models:")
for name in models.keys():
    print(f"   • {name}")

# Configure cross-validation strategy
# Use TimeSeriesSplit for financial data
cv_strategy = TimeSeriesSplit(n_splits=5)
print(f"\n📊 Cross-validation: TimeSeriesSplit with 5 folds")
print(f"   • Preserves temporal order in financial data")
print(f"   • Avoids look-ahead bias")

🤖 MODEL CONFIGURATION
✅ Configured 7 baseline models:
   • Ridge
   • Lasso
   • ElasticNet
   • RandomForest
   • GradientBoosting
   • XGBoost
   • LightGBM

📊 Cross-validation: TimeSeriesSplit with 5 folds
   • Preserves temporal order in financial data
   • Avoids look-ahead bias


In [6]:
# Helper functions for evaluation
def calculate_hit_rate(y_true, y_pred):
    """Calculate hit rate (directional accuracy)"""
    return np.mean(np.sign(y_true) == np.sign(y_pred))

def calculate_sharpe_ratio(returns, risk_free_rate=0.0):
    """Calculate Sharpe ratio"""
    excess_returns = returns - risk_free_rate
    if np.std(excess_returns) == 0:
        return 0.0
    return np.mean(excess_returns) / np.std(excess_returns) * np.sqrt(252)

def evaluate_model(model, X_train, y_train, X_test, y_test, model_name="Model"):
    """Comprehensive model evaluation"""
    try:
        # Train model
        model.fit(X_train, y_train)
        
        # Predictions
        y_pred_train = model.predict(X_train)
        y_pred_test = model.predict(X_test)
        
        # Calculate metrics
        results = {
            'model': model_name,
            'train_r2': r2_score(y_train, y_pred_train),
            'test_r2': r2_score(y_test, y_pred_test),
            'train_rmse': np.sqrt(mean_squared_error(y_train, y_pred_train)),
            'test_rmse': np.sqrt(mean_squared_error(y_test, y_pred_test)),
            'hit_rate': calculate_hit_rate(y_test, y_pred_test),
            'sharpe_ratio': calculate_sharpe_ratio(y_pred_test),
            'overfitting': abs(results.get('train_r2', 0) - results.get('test_r2', 0)) if 'train_r2' in locals() else 0
        }
        
        # Fix overfitting calculation
        results['overfitting'] = abs(results['train_r2'] - results['test_r2'])
        
        return results
        
    except Exception as e:
        return {
            'model': model_name,
            'error': str(e),
            'train_r2': np.nan,
            'test_r2': np.nan,
            'train_rmse': np.nan,
            'test_rmse': np.nan,
            'hit_rate': np.nan,
            'sharpe_ratio': np.nan,
            'overfitting': np.nan
        }

print("✅ Evaluation functions defined")

✅ Evaluation functions defined


## 4. Model Training and Selection

In [7]:
# Main model selection and training loop
print("🚀 MODEL SELECTION AND TRAINING")
print("=" * 60)

if primary_target is None:
    print("❌ No target variable found! Cannot proceed with training.")
else:
    # Prepare target variable
    y_train_full = df_train[primary_target].copy()
    
    # Store all results
    all_results = []
    best_model_info = {'score': -np.inf, 'model': None, 'features': None, 'scaler': None}
    
    print(f"🎯 Target variable: {primary_target}")
    print(f"   • Training samples: {len(y_train_full):,}")
    print(f"   • Target missing values: {y_train_full.isnull().sum()}")
    
    # Remove rows with missing targets
    valid_idx = ~y_train_full.isnull()
    if valid_idx.sum() < len(y_train_full):
        print(f"   • Removing {(~valid_idx).sum()} samples with missing targets")
        df_train_clean = df_train[valid_idx].copy()
        y_train_full = y_train_full[valid_idx].copy()
    else:
        df_train_clean = df_train.copy()
    
    print(f"   • Final training samples: {len(y_train_full):,}")
    
    # Test each feature set
    for set_name, feature_list in feature_sets.items():
        print(f"\n📊 TESTING FEATURE SET: {set_name.upper()}")
        print("-" * 50)
        
        try:
            # Get features that exist in both train and test
            available_features = [f for f in feature_list if f in df_train_clean.columns and f in df_test.columns]
            
            if len(available_features) == 0:
                print(f"   ❌ No valid features found for {set_name}")
                continue
                
            print(f"   • Features: {len(available_features)} available")
            
            # Prepare feature data
            X_train_raw = df_train_clean[available_features].copy()
            X_test_raw = df_test[available_features].copy()
            
            # Handle missing values with simple imputation
            imputer = SimpleImputer(strategy='median')
            X_train_imputed = pd.DataFrame(
                imputer.fit_transform(X_train_raw),
                columns=available_features,
                index=X_train_raw.index
            )
            X_test_imputed = pd.DataFrame(
                imputer.transform(X_test_raw),
                columns=available_features,
                index=X_test_raw.index
            )
            
            # Check for remaining missing values
            train_missing = X_train_imputed.isnull().sum().sum()
            test_missing = X_test_imputed.isnull().sum().sum()
            
            if train_missing > 0 or test_missing > 0:
                print(f"   ⚠️ Remaining missing values: Train={train_missing}, Test={test_missing}")
                # Forward fill any remaining missing values
                X_train_imputed = X_train_imputed.fillna(method='ffill').fillna(0)
                X_test_imputed = X_test_imputed.fillna(method='ffill').fillna(0)
            
            # Scale features
            scaler = RobustScaler()
            X_train_scaled = scaler.fit_transform(X_train_imputed)
            X_test_scaled = scaler.transform(X_test_imputed)
            
            # Create train/validation split maintaining time order
            split_idx = int(len(X_train_scaled) * 0.8)
            
            X_train_split = X_train_scaled[:split_idx]
            X_val_split = X_train_scaled[split_idx:]
            y_train_split = y_train_full.iloc[:split_idx]
            y_val_split = y_train_full.iloc[split_idx:]
            
            print(f"   • Train split: {len(X_train_split):,} samples")
            print(f"   • Validation split: {len(X_val_split):,} samples")
            
            # Test each model
            set_results = []
            for model_name, model in models.items():
                print(f"   🔄 {model_name}...", end=" ")
                
                result = evaluate_model(
                    model, X_train_split, y_train_split, 
                    X_val_split, y_val_split, model_name
                )
                result['feature_set'] = set_name
                result['n_features'] = len(available_features)
                
                if 'error' in result:
                    print(f"❌ Failed: {result['error'][:50]}")
                else:
                    print(f"✅ R²={result['test_r2']:.4f}, Hit={result['hit_rate']:.3f}")
                    
                    # Track best model
                    if result['test_r2'] > best_model_info['score']:
                        best_model_info.update({
                            'score': result['test_r2'],
                            'model': model,
                            'model_name': model_name,
                            'features': available_features,
                            'scaler': scaler,
                            'feature_set': set_name
                        })
                
                set_results.append(result)
                all_results.append(result)
            
            # Summary for this feature set
            valid_results = [r for r in set_results if 'error' not in r]
            if valid_results:
                best_in_set = max(valid_results, key=lambda x: x['test_r2'])
                print(f"   🏆 Best in set: {best_in_set['model']} (R²={best_in_set['test_r2']:.4f})")
            
        except Exception as e:
            print(f"   ❌ Feature set {set_name} failed: {str(e)[:100]}")
    
    print(f"\n✅ Model selection complete! Tested {len(all_results)} model configurations.")

🚀 MODEL SELECTION AND TRAINING
🎯 Target variable: forward_returns
   • Training samples: 8,990
   • Target missing values: 0
   • Final training samples: 8,990

📊 TESTING FEATURE SET: COMPLETE_ONLY
--------------------------------------------------
   • Features: 9 available
   • Train split: 7,192 samples
   • Validation split: 1,798 samples
   🔄 Ridge... ✅ R²=-0.0020, Hit=0.516
   🔄 Lasso... ✅ R²=-0.0003, Hit=0.552
   🔄 ElasticNet... ✅ R²=-0.0003, Hit=0.552
   🔄 RandomForest... ✅ R²=-0.0039, Hit=0.538
   🔄 GradientBoosting... ✅ R²=-0.0039, Hit=0.538
   🔄 GradientBoosting... ✅ R²=-0.0036, Hit=0.536
   🔄 XGBoost... ✅ R²=-0.0029, Hit=0.537
   🔄 LightGBM... ✅ R²=-0.0036, Hit=0.536
   🔄 XGBoost... ✅ R²=-0.0029, Hit=0.537
   🔄 LightGBM... ✅ R²=-0.0039, Hit=0.537
   🏆 Best in set: Lasso (R²=-0.0003)

📊 TESTING FEATURE SET: RELIABLE
--------------------------------------------------
   • Features: 9 available
   • Train split: 7,192 samples
   • Validation split: 1,798 samples
   🔄 Ridge... 

## 5. Results Analysis and Best Model Selection

In [8]:
# Analyze results and select best model
print("📊 RESULTS ANALYSIS")
print("=" * 60)

if len(all_results) == 0:
    print("❌ No successful model runs to analyze!")
else:
    # Create results DataFrame
    results_df = pd.DataFrame([r for r in all_results if 'error' not in r])
    
    if len(results_df) == 0:
        print("❌ All models failed! Check data and model configurations.")
    else:
        print(f"✅ Successfully trained {len(results_df)} models")
        
        # Overall performance summary
        print(f"\n📈 PERFORMANCE SUMMARY:")
        print(f"   • Average R²: {results_df['test_r2'].mean():.4f}")
        print(f"   • Best R²: {results_df['test_r2'].max():.4f}")
        print(f"   • Worst R²: {results_df['test_r2'].min():.4f}")
        print(f"   • Average Hit Rate: {results_df['hit_rate'].mean():.3f}")
        print(f"   • Best Hit Rate: {results_df['hit_rate'].max():.3f}")
        
        # Top 5 models
        print(f"\n🏆 TOP 5 MODELS:")
        print("-" * 40)
        top_models = results_df.nlargest(5, 'test_r2')
        
        for i, (_, row) in enumerate(top_models.iterrows(), 1):
            print(f"   {i}. {row['model']} ({row['feature_set']})")
            print(f"      R² = {row['test_r2']:.4f}, Hit Rate = {row['hit_rate']:.3f}")
            print(f"      Features: {row['n_features']}, RMSE: {row['test_rmse']:.6f}")
            print(f"      Overfitting: {row['overfitting']:.4f}")
            print()
        
        # Model comparison
        print(f"📊 MODEL TYPE COMPARISON:")
        print("-" * 30)
        model_comparison = results_df.groupby('model').agg({
            'test_r2': ['mean', 'max', 'count'],
            'hit_rate': 'mean',
            'overfitting': 'mean'
        }).round(4)
        
        model_comparison.columns = ['avg_r2', 'best_r2', 'count', 'avg_hit_rate', 'avg_overfitting']
        model_comparison = model_comparison.sort_values('avg_r2', ascending=False)
        
        for model_name, stats in model_comparison.iterrows():
            print(f"   {model_name}:")
            print(f"      Avg R²: {stats['avg_r2']:.4f} (Best: {stats['best_r2']:.4f})")
            print(f"      Avg Hit Rate: {stats['avg_hit_rate']:.3f}")
            print(f"      Avg Overfitting: {stats['avg_overfitting']:.4f}")
            print(f"      Configurations: {int(stats['count'])}")
            print()
        
        # Feature set comparison
        print(f"🔧 FEATURE SET COMPARISON:")
        print("-" * 30)
        feature_comparison = results_df.groupby('feature_set').agg({
            'test_r2': ['mean', 'max'],
            'hit_rate': 'mean',
            'n_features': 'first'
        }).round(4)
        
        feature_comparison.columns = ['avg_r2', 'best_r2', 'avg_hit_rate', 'n_features']
        feature_comparison = feature_comparison.sort_values('avg_r2', ascending=False)
        
        for set_name, stats in feature_comparison.iterrows():
            print(f"   {set_name} ({int(stats['n_features'])} features):")
            print(f"      Avg R²: {stats['avg_r2']:.4f} (Best: {stats['best_r2']:.4f})")
            print(f"      Avg Hit Rate: {stats['avg_hit_rate']:.3f}")
            print()

📊 RESULTS ANALYSIS
✅ Successfully trained 28 models

📈 PERFORMANCE SUMMARY:
   • Average R²: -0.0762
   • Best R²: -0.0003
   • Worst R²: -0.7524
   • Average Hit Rate: 0.532
   • Best Hit Rate: 0.552

🏆 TOP 5 MODELS:
----------------------------------------
   1. Lasso (complete_only)
      R² = -0.0003, Hit Rate = 0.552
      Features: 9, RMSE: 0.011091
      Overfitting: 0.0003

   2. ElasticNet (complete_only)
      R² = -0.0003, Hit Rate = 0.552
      Features: 9, RMSE: 0.011091
      Overfitting: 0.0003

   3. Lasso (reliable)
      R² = -0.0003, Hit Rate = 0.552
      Features: 9, RMSE: 0.011091
      Overfitting: 0.0003

   4. ElasticNet (reliable)
      R² = -0.0003, Hit Rate = 0.552
      Features: 9, RMSE: 0.011091
      Overfitting: 0.0003

   5. Lasso (extended)
      R² = -0.0003, Hit Rate = 0.552
      Features: 86, RMSE: 0.011091
      Overfitting: 0.0003

📊 MODEL TYPE COMPARISON:
------------------------------
   ElasticNet:
      Avg R²: -0.0003 (Best: -0.0003)
      

## 6. Final Model Training and Test Set Evaluation

In [9]:
# Train best model on full training data and evaluate on test set
print("🎯 FINAL MODEL TRAINING AND TEST EVALUATION")
print("=" * 60)

if best_model_info['model'] is None:
    print("❌ No best model found! Cannot proceed with final evaluation.")
else:
    print(f"🏆 Selected Best Model: {best_model_info['model_name']}")
    print(f"   • Feature Set: {best_model_info['feature_set']}")
    print(f"   • Features: {len(best_model_info['features'])}")
    print(f"   • Validation R²: {best_model_info['score']:.4f}")
    
    try:
        # Prepare final training data
        X_train_final = df_train_clean[best_model_info['features']].copy()
        y_train_final = y_train_full.copy()
        
        # Prepare test data
        X_test_final = df_test[best_model_info['features']].copy()
        
        print(f"\n📊 Final Data Preparation:")
        print(f"   • Training samples: {len(X_train_final):,}")
        print(f"   • Test samples: {len(X_test_final):,}")
        print(f"   • Features: {len(best_model_info['features'])}")
        
        # Apply same preprocessing pipeline
        imputer = SimpleImputer(strategy='median')
        X_train_imputed = pd.DataFrame(
            imputer.fit_transform(X_train_final),
            columns=best_model_info['features']
        )
        X_test_imputed = pd.DataFrame(
            imputer.transform(X_test_final),
            columns=best_model_info['features']
        )
        
        # Handle any remaining missing values
        X_train_imputed = X_train_imputed.fillna(method='ffill').fillna(0)
        X_test_imputed = X_test_imputed.fillna(method='ffill').fillna(0)
        
        # Scale features
        scaler = RobustScaler()
        X_train_scaled = scaler.fit_transform(X_train_imputed)
        X_test_scaled = scaler.transform(X_test_imputed)
        
        print(f"   • Preprocessing complete")
        
        # Train final model on full training data
        final_model = best_model_info['model']
        print(f"\n🚀 Training final model...")
        final_model.fit(X_train_scaled, y_train_final)
        
        # Make predictions
        train_predictions = final_model.predict(X_train_scaled)
        test_predictions = final_model.predict(X_test_scaled)
        
        # Calculate final metrics
        final_train_r2 = r2_score(y_train_final, train_predictions)
        final_train_rmse = np.sqrt(mean_squared_error(y_train_final, train_predictions))
        final_train_hit_rate = calculate_hit_rate(y_train_final, train_predictions)
        
        print(f"\n✅ FINAL MODEL PERFORMANCE:")
        print(f"=" * 40)
        print(f"📊 Training Set Performance:")
        print(f"   • R² Score: {final_train_r2:.4f}")
        print(f"   • RMSE: {final_train_rmse:.6f}")
        print(f"   • Hit Rate: {final_train_hit_rate:.3f}")
        print(f"   • Sharpe Ratio: {calculate_sharpe_ratio(train_predictions):.4f}")
        
        print(f"\n🎯 Test Set Predictions Generated:")
        print(f"   • Number of predictions: {len(test_predictions):,}")
        print(f"   • Prediction range: [{test_predictions.min():.6f}, {test_predictions.max():.6f}]")
        print(f"   • Mean prediction: {test_predictions.mean():.6f}")
        print(f"   • Std prediction: {test_predictions.std():.6f}")
        
        # Create submission DataFrame
        submission_df = pd.DataFrame({
            'date_id': df_test['date_id'],
            'forward_returns_prediction': test_predictions
        })
        
        # Save predictions
        submission_path = '../data/predictions/baseline_predictions.csv'
        import os
        os.makedirs('../data/predictions', exist_ok=True)
        submission_df.to_csv(submission_path, index=False)
        
        print(f"\n💾 Predictions saved to: {submission_path}")
        print(f"   • Ready for competition submission")
        
        # Display sample predictions
        print(f"\n📋 Sample Predictions:")
        print(submission_df.head(10).to_string(index=False))
        
    except Exception as e:
        print(f"❌ Final model training failed: {str(e)}")
        import traceback
        traceback.print_exc()

🎯 FINAL MODEL TRAINING AND TEST EVALUATION
🏆 Selected Best Model: Lasso
   • Feature Set: complete_only
   • Features: 9
   • Validation R²: -0.0003

📊 Final Data Preparation:
   • Training samples: 8,990
   • Test samples: 10
   • Features: 9
   • Preprocessing complete

🚀 Training final model...

✅ FINAL MODEL PERFORMANCE:
📊 Training Set Performance:
   • R² Score: 0.0000
   • RMSE: 0.010550
   • Hit Rate: 0.539
   • Sharpe Ratio: 68699043599706112.0000

🎯 Test Set Predictions Generated:
   • Number of predictions: 10
   • Prediction range: [0.000469, 0.000469]
   • Mean prediction: 0.000469
   • Std prediction: 0.000000

💾 Predictions saved to: ../data/predictions/baseline_predictions.csv
   • Ready for competition submission

📋 Sample Predictions:
 date_id  forward_returns_prediction
    8980                    0.000469
    8981                    0.000469
    8982                    0.000469
    8983                    0.000469
    8984                    0.000469
    8985        

## 7. Summary and Next Steps

In [10]:
# Final summary
print("📋 BASELINE MODEL SUMMARY")
print("=" * 60)

if 'results_df' in locals() and len(results_df) > 0:
    print(f"🎯 Model Selection Results:")
    print(f"   • Total configurations tested: {len(all_results)}")
    print(f"   • Successful runs: {len(results_df)}")
    print(f"   • Best validation R²: {results_df['test_r2'].max():.4f}")
    print(f"   • Best hit rate: {results_df['hit_rate'].max():.3f}")
    
    if 'final_train_r2' in locals():
        print(f"\n🏆 Final Model Performance:")
        print(f"   • Model: {best_model_info['model_name']}")
        print(f"   • Feature Set: {best_model_info['feature_set']}")
        print(f"   • Training R²: {final_train_r2:.4f}")
        print(f"   • Training Hit Rate: {final_train_hit_rate:.3f}")
        print(f"   • Test predictions: ✅ Generated")
    
    print(f"\n📊 Key Insights:")
    best_model_type = results_df.loc[results_df['test_r2'].idxmax(), 'model']
    best_feature_set = results_df.loc[results_df['test_r2'].idxmax(), 'feature_set']
    print(f"   • Best performing model type: {best_model_type}")
    print(f"   • Best performing feature set: {best_feature_set}")
    print(f"   • Average overfitting: {results_df['overfitting'].mean():.4f}")
    
    # Performance interpretation
    avg_r2 = results_df['test_r2'].mean()
    if avg_r2 > 0.1:
        performance_rating = "🟢 Good"
    elif avg_r2 > 0.0:
        performance_rating = "🟡 Moderate"
    else:
        performance_rating = "🔴 Poor - Consider feature engineering"
    
    print(f"   • Overall performance: {performance_rating}")
    
else:
    print("❌ No successful model runs to summarize")

print(f"\n🚀 Recommended Next Steps:")
print(f"   1. 🔍 Feature Engineering: Create interaction terms, lags, rolling statistics")
print(f"   2. 🎯 Hyperparameter Tuning: Optimize best performing models")
print(f"   3. 🔄 Ensemble Methods: Combine multiple models for better performance")
print(f"   4. 📊 Advanced Missing Value Handling: Use domain-specific imputation")
print(f"   5. ⚖️ Model Interpretability: Analyze feature importance and predictions")

print(f"\n" + "=" * 60)
print(f"✅ BASELINE MODEL PIPELINE COMPLETE")
print(f"📈 READY FOR ADVANCED MODELING PHASE")
print(f"=" * 60)

📋 BASELINE MODEL SUMMARY
🎯 Model Selection Results:
   • Total configurations tested: 28
   • Successful runs: 28
   • Best validation R²: -0.0003
   • Best hit rate: 0.552

🏆 Final Model Performance:
   • Model: Lasso
   • Feature Set: complete_only
   • Training R²: 0.0000
   • Training Hit Rate: 0.539
   • Test predictions: ✅ Generated

📊 Key Insights:
   • Best performing model type: Lasso
   • Best performing feature set: complete_only
   • Average overfitting: 0.2023
   • Overall performance: 🔴 Poor - Consider feature engineering

🚀 Recommended Next Steps:
   1. 🔍 Feature Engineering: Create interaction terms, lags, rolling statistics
   2. 🎯 Hyperparameter Tuning: Optimize best performing models
   3. 🔄 Ensemble Methods: Combine multiple models for better performance
   4. 📊 Advanced Missing Value Handling: Use domain-specific imputation
   5. ⚖️ Model Interpretability: Analyze feature importance and predictions

✅ BASELINE MODEL PIPELINE COMPLETE
📈 READY FOR ADVANCED MODELING P

In [11]:
## 8. Performance Comparison with Clean Imputed Data

#Now let's test the same best model using the clean `train_imputed.csv` dataset to measure performance improvement from advanced missing value handling.

In [12]:
# Load the clean imputed training data and compare performance
print("🧹 PERFORMANCE COMPARISON WITH CLEAN IMPUTED DATA")
print("=" * 70)

try:
    # Load the clean imputed dataset
    df_train_imputed = pd.read_csv('../data/cleaned/train_imputed.csv')
    print(f"✅ Clean imputed data loaded successfully:")
    print(f"   • Shape: {df_train_imputed.shape}")
    print(f"   • Missing values: {df_train_imputed.isnull().sum().sum():,}")
    
    # Store original baseline results for comparison
    original_r2 = best_model_info['score']
    original_model_name = best_model_info['model_name']
    original_feature_set = best_model_info['feature_set']
    
    print(f"\n📊 ORIGINAL BASELINE PERFORMANCE:")
    print(f"   • Model: {original_model_name}")
    print(f"   • Feature Set: {original_feature_set}")
    print(f"   • Validation R²: {original_r2:.4f}")
    print(f"   • Training R²: {final_train_r2:.4f}")
    print(f"   • Hit Rate: {final_train_hit_rate:.3f}")
    
    # Prepare clean data with same target
    y_imputed = df_train_imputed[primary_target].copy()
    print(f"\n🎯 Clean Data Target Analysis:")
    print(f"   • Target: {primary_target}")
    print(f"   • Samples: {len(y_imputed):,}")
    print(f"   • Missing values: {y_imputed.isnull().sum()}")
    
    # Remove any rows with missing targets in clean data
    valid_idx_clean = ~y_imputed.isnull()
    if valid_idx_clean.sum() < len(y_imputed):
        print(f"   • Removing {(~valid_idx_clean).sum()} samples with missing targets")
        df_train_imputed_clean = df_train_imputed[valid_idx_clean].copy()
        y_imputed = y_imputed[valid_idx_clean].copy()
    else:
        df_train_imputed_clean = df_train_imputed.copy()
    
    print(f"   • Final clean samples: {len(y_imputed):,}")
    
except FileNotFoundError:
    print("❌ Clean imputed data file not found!")
    print("   • Please ensure train_imputed.csv exists in ../data/cleaned/")
    print("   • Run the advanced missing value handling notebook first")
    df_train_imputed_clean = None

if 'df_train_imputed_clean' in locals() and df_train_imputed_clean is not None:
    print(f"\n🔄 TESTING BEST MODEL WITH CLEAN DATA...")
    print("-" * 50)
    
    # Test different feature strategies with clean data
    clean_results = []
    
    # Strategy 1: Use same features as original best model
    print(f"📊 Strategy 1: Same features as original best model")
    try:
        # Get the same features used in original best model
        original_features = best_model_info['features']
        available_clean_features = [f for f in original_features if f in df_train_imputed_clean.columns]
        
        if len(available_clean_features) > 0:
            print(f"   • Using {len(available_clean_features)} features from original best model")
            
            # Prepare clean feature data
            X_clean = df_train_imputed_clean[available_clean_features].copy()
            
            # Check for missing values in clean data
            clean_missing = X_clean.isnull().sum().sum()
            print(f"   • Missing values in features: {clean_missing}")
            
            if clean_missing > 0:
                # Apply same imputation as before for consistency
                imputer_clean = SimpleImputer(strategy='median')
                X_clean_imputed = pd.DataFrame(
                    imputer_clean.fit_transform(X_clean),
                    columns=available_clean_features
                )
                X_clean_imputed = X_clean_imputed.fillna(method='ffill').fillna(0)
            else:
                X_clean_imputed = X_clean.copy()
            
            # Scale features
            scaler_clean = RobustScaler()
            X_clean_scaled = scaler_clean.fit_transform(X_clean_imputed)
            
            # Create train/validation split maintaining time order
            split_idx_clean = int(len(X_clean_scaled) * 0.8)
            
            X_train_clean_split = X_clean_scaled[:split_idx_clean]
            X_val_clean_split = X_clean_scaled[split_idx_clean:]
            y_train_clean_split = y_imputed.iloc[:split_idx_clean]
            y_val_clean_split = y_imputed.iloc[split_idx_clean:]
            
            print(f"   • Train split: {len(X_train_clean_split):,} samples")
            print(f"   • Validation split: {len(X_val_clean_split):,} samples")
            
            # Test the same model type as original best
            from sklearn.base import clone
            clean_model = clone(best_model_info['model'])
            
            # Evaluate model with clean data
            clean_result = evaluate_model(
                clean_model, X_train_clean_split, y_train_clean_split,
                X_val_clean_split, y_val_clean_split, f"{original_model_name}_Clean"
            )
            clean_result['strategy'] = 'same_features'
            clean_result['n_features'] = len(available_clean_features)
            clean_results.append(clean_result)
            
            print(f"   ✅ {original_model_name} with clean data:")
            print(f"      R² = {clean_result['test_r2']:.4f}")
            print(f"      Hit Rate = {clean_result['hit_rate']:.3f}")
            print(f"      RMSE = {clean_result['test_rmse']:.6f}")
            
        else:
            print(f"   ❌ No matching features found in clean data")
            
    except Exception as e:
        print(f"   ❌ Strategy 1 failed: {str(e)[:100]}")
    
    # Strategy 2: Use all available features from clean data (no missing values)
    print(f"\n📊 Strategy 2: Use all available clean features")
    try:
        # Get all feature columns from clean data (excluding targets and IDs)
        clean_feature_cols = [col for col in df_train_imputed_clean.columns if col not in exclude_cols]
        
        if len(clean_feature_cols) > 0:
            print(f"   • Using {len(clean_feature_cols)} clean features")
            
            # Prepare feature data
            X_all_clean = df_train_imputed_clean[clean_feature_cols].copy()
            
            # Check for missing values
            all_clean_missing = X_all_clean.isnull().sum().sum()
            print(f"   • Missing values: {all_clean_missing}")
            
            if all_clean_missing > 0:
                # Light imputation for any remaining missing values
                imputer_all = SimpleImputer(strategy='median')
                X_all_clean_processed = pd.DataFrame(
                    imputer_all.fit_transform(X_all_clean),
                    columns=clean_feature_cols
                )
                X_all_clean_processed = X_all_clean_processed.fillna(method='ffill').fillna(0)
            else:
                X_all_clean_processed = X_all_clean.copy()
            
            # Scale features
            scaler_all = RobustScaler()
            X_all_clean_scaled = scaler_all.fit_transform(X_all_clean_processed)
            
            # Create train/validation split
            split_idx_all = int(len(X_all_clean_scaled) * 0.8)
            
            X_train_all_split = X_all_clean_scaled[:split_idx_all]
            X_val_all_split = X_all_clean_scaled[split_idx_all:]
            y_train_all_split = y_imputed.iloc[:split_idx_all]
            y_val_all_split = y_imputed.iloc[split_idx_all:]
            
            print(f"   • Train split: {len(X_train_all_split):,} samples")
            print(f"   • Validation split: {len(X_val_all_split):,} samples")
            
            # Test same model type with all clean features
            clean_model_all = clone(best_model_info['model'])
            
            clean_result_all = evaluate_model(
                clean_model_all, X_train_all_split, y_train_all_split,
                X_val_all_split, y_val_all_split, f"{original_model_name}_AllClean"
            )
            clean_result_all['strategy'] = 'all_clean_features'
            clean_result_all['n_features'] = len(clean_feature_cols)
            clean_results.append(clean_result_all)
            
            print(f"   ✅ {original_model_name} with all clean features:")
            print(f"      R² = {clean_result_all['test_r2']:.4f}")
            print(f"      Hit Rate = {clean_result_all['hit_rate']:.3f}")
            print(f"      RMSE = {clean_result_all['test_rmse']:.6f}")
            
        else:
            print(f"   ❌ No feature columns found in clean data")
            
    except Exception as e:
        print(f"   ❌ Strategy 2 failed: {str(e)[:100]}")
    
    # Performance comparison summary
    print(f"\n📈 PERFORMANCE IMPROVEMENT ANALYSIS")
    print("=" * 60)
    
    if len(clean_results) > 0:
        # Best clean result
        best_clean = max(clean_results, key=lambda x: x['test_r2'])
        
        print(f"🏆 BEST CLEAN DATA PERFORMANCE:")
        print(f"   • Model: {best_clean['model']}")
        print(f"   • Strategy: {best_clean['strategy']}")
        print(f"   • Features: {best_clean['n_features']}")
        print(f"   • R² Score: {best_clean['test_r2']:.4f}")
        print(f"   • Hit Rate: {best_clean['hit_rate']:.3f}")
        print(f"   • RMSE: {best_clean['test_rmse']:.6f}")
        
        print(f"\n📊 IMPROVEMENT METRICS:")
        r2_improvement = best_clean['test_r2'] - original_r2
        hit_rate_improvement = best_clean['hit_rate'] - final_train_hit_rate
        
        print(f"   • R² Improvement: {r2_improvement:+.4f}")
        print(f"   • Hit Rate Improvement: {hit_rate_improvement:+.3f}")
        
        if r2_improvement > 0:
            relative_improvement = (r2_improvement / abs(original_r2)) * 100 if original_r2 != 0 else float('inf')
            print(f"   • Relative R² Improvement: {relative_improvement:+.1f}%")
        else:
            print(f"   • Relative R² Improvement: {r2_improvement/abs(original_r2)*100:+.1f}%" if original_r2 != 0 else "N/A")
        
        # Performance interpretation
        if r2_improvement > 0.01:
            improvement_status = "🟢 Significant Improvement"
        elif r2_improvement > 0:
            improvement_status = "🟡 Modest Improvement"
        else:
            improvement_status = "🔴 No Improvement"
        
        print(f"   • Status: {improvement_status}")
        
        # All strategies comparison
        if len(clean_results) > 1:
            print(f"\n🔄 ALL STRATEGIES COMPARISON:")
            for i, result in enumerate(clean_results, 1):
                print(f"   {i}. {result['strategy']} ({result['n_features']} features):")
                print(f"      R² = {result['test_r2']:.4f}, Hit Rate = {result['hit_rate']:.3f}")
        
        # Store best clean model info for potential final training
        best_clean_model_info = {
            'model': best_clean['model'] if 'model' not in best_clean or isinstance(best_clean['model'], str) else clone(best_model_info['model']),
            'strategy': best_clean['strategy'],
            'features': clean_feature_cols if best_clean['strategy'] == 'all_clean_features' else available_clean_features,
            'score': best_clean['test_r2'],
            'data': 'train_imputed.csv'
        }
        
        print(f"\n✅ Clean data testing complete!")
        print(f"   • Advanced missing value handling shows {improvement_status.split()[-1].lower()}")
        
    else:
        print(f"❌ No clean data results to compare")
        
else:
    print(f"\n⚠️ Skipping clean data comparison - imputed dataset not available")

🧹 PERFORMANCE COMPARISON WITH CLEAN IMPUTED DATA
✅ Clean imputed data loaded successfully:
   • Shape: (8990, 100)
   • Missing values: 110,204

📊 ORIGINAL BASELINE PERFORMANCE:
   • Model: Lasso
   • Feature Set: complete_only
   • Validation R²: -0.0003
   • Training R²: 0.0000
   • Hit Rate: 0.539

🎯 Clean Data Target Analysis:
   • Target: forward_returns
   • Samples: 8,990
   • Missing values: 0
   • Final clean samples: 8,990

🔄 TESTING BEST MODEL WITH CLEAN DATA...
--------------------------------------------------
📊 Strategy 1: Same features as original best model
   • Using 9 features from original best model
   • Missing values in features: 0
   • Train split: 7,192 samples
   • Validation split: 1,798 samples
   ✅ Lasso with clean data:
      R² = -0.0003
      Hit Rate = 0.552
      RMSE = 0.011091

📊 Strategy 2: Use all available clean features
   • Using 96 clean features
   • Missing values: 110204
✅ Clean imputed data loaded successfully:
   • Shape: (8990, 100)
   • M

In [ ]:
# Optional: Train final model with clean data and generate improved test predictions
print("🎯 FINAL MODEL TRAINING WITH CLEAN DATA")
print("=" * 60)

if 'best_clean_model_info' in locals() and 'df_train_imputed_clean' in locals():
    # Check if clean data shows improvement
    if best_clean_model_info['score'] > original_r2:
        print(f"🚀 Clean data shows improvement - training final model with clean data")
        print(f"   • Improvement: {best_clean_model_info['score'] - original_r2:+.4f} R²")
        
        try:
            # Prepare final clean training data
            X_final_clean = df_train_imputed_clean[best_clean_model_info['features']].copy()
            y_final_clean = y_imputed.copy()
            
            # Prepare test data with same features
            test_features_available = [f for f in best_clean_model_info['features'] if f in df_test.columns]
            X_test_final_clean = df_test[test_features_available].copy()
            
            print(f"\n📊 Final Clean Data Preparation:")
            print(f"   • Training samples: {len(X_final_clean):,}")
            print(f"   • Test samples: {len(X_test_final_clean):,}")
            print(f"   • Features: {len(test_features_available)}")
            
            # Handle any missing values in clean data
            clean_missing_final = X_final_clean.isnull().sum().sum()
            test_missing_final = X_test_final_clean.isnull().sum().sum()
            
            if clean_missing_final > 0 or test_missing_final > 0:
                print(f"   • Handling missing values: Train={clean_missing_final}, Test={test_missing_final}")
                imputer_final = SimpleImputer(strategy='median')
                X_final_clean = pd.DataFrame(
                    imputer_final.fit_transform(X_final_clean),
                    columns=best_clean_model_info['features']
                )
                X_test_final_clean = pd.DataFrame(
                    imputer_final.transform(X_test_final_clean),
                    columns=test_features_available
                )
                # Handle any remaining missing values
                X_final_clean = X_final_clean.fillna(method='ffill').fillna(0)
                X_test_final_clean = X_test_final_clean.fillna(method='ffill').fillna(0)
            
            # Scale features
            scaler_final = RobustScaler()
            X_final_clean_scaled = scaler_final.fit_transform(X_final_clean)
            X_test_final_clean_scaled = scaler_final.transform(X_test_final_clean)
            
            print(f"   • Preprocessing complete")
            
            # Train final model on full clean training data
            final_clean_model = clone(best_model_info['model'])
            print(f"\n🚀 Training final clean model...")
            final_clean_model.fit(X_final_clean_scaled, y_final_clean)
            
            # Make predictions
            clean_train_predictions = final_clean_model.predict(X_final_clean_scaled)
            clean_test_predictions = final_clean_model.predict(X_test_final_clean_scaled)
            
            # Calculate final clean metrics
            final_clean_r2 = r2_score(y_final_clean, clean_train_predictions)
            final_clean_rmse = np.sqrt(mean_squared_error(y_final_clean, clean_train_predictions))
            final_clean_hit_rate = calculate_hit_rate(y_final_clean, clean_train_predictions)
            
            print(f"\n✅ FINAL CLEAN MODEL PERFORMANCE:")
            print(f"=" * 50)
            print(f"📊 Training Set Performance:")
            print(f"   • R² Score: {final_clean_r2:.4f}")
            print(f"   • RMSE: {final_clean_rmse:.6f}")
            print(f"   • Hit Rate: {final_clean_hit_rate:.3f}")
            print(f"   • Sharpe Ratio: {calculate_sharpe_ratio(clean_train_predictions):.4f}")
            
            print(f"\n🎯 Clean Test Set Predictions Generated:")
            print(f"   • Number of predictions: {len(clean_test_predictions):,}")
            print(f"   • Prediction range: [{clean_test_predictions.min():.6f}, {clean_test_predictions.max():.6f}]")
            print(f"   • Mean prediction: {clean_test_predictions.mean():.6f}")
            print(f"   • Std prediction: {clean_test_predictions.std():.6f}")
            
            # Create improved submission DataFrame
            improved_submission_df = pd.DataFrame({
                'date_id': df_test['date_id'],
                'forward_returns_prediction': clean_test_predictions
            })
            
            # Save improved predictions
            improved_submission_path = '../data/predictions/baseline_predictions_improved.csv'
            improved_submission_df.to_csv(improved_submission_path, index=False)
            
            print(f"\n💾 Improved predictions saved to: {improved_submission_path}")
            print(f"   • Ready for competition submission")
            
            # Compare predictions
            print(f"\n📋 Prediction Comparison (First 10 samples):")
            comparison_df = pd.DataFrame({
                'date_id': df_test['date_id'].head(10),
                'original_prediction': test_predictions[:10],
                'improved_prediction': clean_test_predictions[:10],
                'difference': clean_test_predictions[:10] - test_predictions[:10]
            })
            print(comparison_df.to_string(index=False, float_format='%.6f'))
            
            # Final improvement summary
            print(f"\n🏆 FINAL IMPROVEMENT SUMMARY:")
            print(f"=" * 50)
            print(f"📈 Training Performance Comparison:")
            print(f"   • Original R²: {final_train_r2:.4f}")
            print(f"   • Improved R²: {final_clean_r2:.4f}")
            print(f"   • R² Improvement: {final_clean_r2 - final_train_r2:+.4f}")
            print(f"   • Original Hit Rate: {final_train_hit_rate:.3f}")
            print(f"   • Improved Hit Rate: {final_clean_hit_rate:.3f}")
            print(f"   • Hit Rate Improvement: {final_clean_hit_rate - final_train_hit_rate:+.3f}")
            
            print(f"\n📊 Test Prediction Analysis:")
            prediction_change = np.abs(clean_test_predictions - test_predictions).mean()
            print(f"   • Average prediction change: {prediction_change:.6f}")
            print(f"   • Max prediction change: {np.abs(clean_test_predictions - test_predictions).max():.6f}")
            
            if final_clean_r2 > final_train_r2:
                improvement_magnitude = "🟢 Significant" if (final_clean_r2 - final_train_r2) > 0.01 else "🟡 Modest"
                print(f"   • Overall improvement: {improvement_magnitude}")
                print(f"   • Recommendation: Use improved predictions for submission")
            else:
                print(f"   • Overall improvement: 🔴 No significant improvement")
                print(f"   • Recommendation: Review feature engineering approach")
            
        except Exception as e:
            print(f"❌ Final clean model training failed: {str(e)}")
            import traceback
            traceback.print_exc()
    
    else:
        print(f"📊 Clean data does not show significant improvement over original baseline")
        print(f"   • Original R²: {original_r2:.4f}")
        print(f"   • Best clean R²: {best_clean_model_info['score']:.4f}")
        print(f"   • Difference: {best_clean_model_info['score'] - original_r2:+.4f}")
        print(f"   • Recommendation: Focus on feature engineering and advanced modeling")

else:
    print(f"⚠️ Clean data comparison not available - using original baseline results")

print(f"\n" + "=" * 60)
print(f"✅ COMPLETE BASELINE VS CLEAN DATA ANALYSIS FINISHED")
print(f"📈 READY FOR ADVANCED FEATURE ENGINEERING")
print(f"=" * 60)